In [16]:
import json
import nltk
import numpy as np

In [ ]:
with open('data/SciQ dataset/train.json', 'r') as rf:
    train = json.load(rf)

In [65]:
def tokenize(sequence):
    tokens = [token.replace("``", '"').replace("''", '"') for token in nltk.word_tokenize(sequence)]
#     return list(map(lambda x:x.encode('utf8'), tokens))
    return tokens

In [ ]:
## Glove stuff - Pass pretrained embeddings as input
filename = 'data/glove.6B.50d.txt'
def loadGloVe(filename):
    rev_vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        rev_vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
    return vocab,rev_vocab,embd
vocab,rev_vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)


In [ ]:
supportTokens = []
questionTokens = []
distractor1Tokens = []
distractor2Tokens = []
distractor3Tokens = []
question_embed = []
UNK_ID = 2
for index in range (len(train)):
#     supportTokens = tokenize(train[index]['support'])
    questionTokens = tokenize(train[index]['question'])
#     distractor1Tokens = tokenize(train[index]['distractor1'])
#     distractor2Tokens = tokenize(train[index]['distractor2'])
#     distractor3Tokens = tokenize(train[index]['distractor3'])
#     answerTokens = tokenize(train[index]['correct_answer'])
    
#     support_ids = [str(vocab.get(w, UNK_ID)) for w in supportTokens]
    question_ids = [str(vocab.get(w, UNK_ID)) for w in questionTokens]
#     distractor1_ids = [str(vocab.get(w, UNK_ID)) for w in distractor1Tokens]
#     distractor2_ids = [str(vocab.get(w, UNK_ID)) for w in distractor2Tokens]
#     distractor3_ids = [str(vocab.get(w, UNK_ID)) for w in distractor3Tokens]
#     answer_ids = [str(vocab.get(w, UNK_ID)) for w in answerTokens]
    
    question_embed.append(tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(question_ids)))))
#     support_embed.append(tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(support_ids)))))
#     answer_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(answerTokens[index]))))
#     distractor1_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(distractor1Tokens[index]))))
#     distractor2_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(distractor2Tokens[index]))))
#     distractor3_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(distractor3Tokens[index]))))

In [67]:
some_list = tokenize(train[0]['support'])
print(some_list)

['Mesophiles', 'grow', 'best', 'in', 'moderate', 'temperature', ',', 'typically', 'between', '25°C', 'and', '40°C', '(', '77°F', 'and', '104°F', ')', '.', 'Mesophiles', 'are', 'often', 'found', 'living', 'in', 'or', 'on', 'the', 'bodies', 'of', 'humans', 'or', 'other', 'animals', '.', 'The', 'optimal', 'growth', 'temperature', 'of', 'many', 'pathogenic', 'mesophiles', 'is', '37°C', '(', '98°F', ')', ',', 'the', 'normal', 'human', 'body', 'temperature', '.', 'Mesophilic', 'organisms', 'have', 'important', 'uses', 'in', 'food', 'preparation', ',', 'including', 'cheese', ',', 'yogurt', ',', 'beer', 'and', 'wine', '.']


In [ ]:
from pprint import pprint as pp
sess = tf.Session()
W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]),
                trainable=False, name="W")
embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
embedding_init = W.assign(embedding_placeholder)
sess.run(embedding_init, feed_dict={embedding_placeholder: embedding})

In [ ]:
# 
from tensorflow.contrib import learn
#init vocab processor
vocab_processor = learn.preprocessing.VocabularyProcessor(len(train))
#fit the vocab from glove
pretrain = vocab_processor.fit(rev_vocab)
#transform inputs
for index in range (len(train)):
    question_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(questionTokens[index]))))
    support_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(supportTokens[index]))))
    answer_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(answerTokens[index]))))
    distractor1_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(distractor1Tokens[index]))))
    distractor2_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(distractor2Tokens[index]))))
    distractor3_embed = tf.nn.embedding_lookup(W, np.array(list(vocab_processor.transform(distractor3Tokens[index]))))

In [12]:
import tensorflow as tf
def setup_word_embeddings(embeddings, question_ids, support_ids):
    '''
        Create an embedding matrix (initialised with pretrained glove vectors and updated only if self.config.train_embeddings is true)
        lookup into this matrix and apply dropout (which is 1 at test time and self.config.dropout at train time)
    '''
    with tf.variable_scope("vocab_embeddings"):
        _word_embeddings = tf.Variable(embeddings, name="_word_embeddings", dtype=tf.float32, trainable= self.config.train_embeddings)
        question_emb = tf.nn.embedding_lookup(_word_embeddings, question_ids, name = "question") # (-1, Q, D)
        passage_emb = tf.nn.embedding_lookup(_word_embeddings, passage_ids, name = "passage") # (-1, P, D)
        # Apply dropout
        self.question = tf.nn.dropout(question_emb, self.dropout)


In [13]:
## Pass tokens (question/answer/support) as input